In [1]:
import os, warnings

warnings.filterwarnings('ignore')

import tensorflow as tf

import keras_tuner as kt

from tensorflow.keras import layers, models, datasets, optimizers

In [2]:
import numpy as np

(train_image, train_label), (test_image, test_label) = datasets.mnist.load_data()

train_image = tf.cast(train_image, tf.float32) / 255.0

np.min(train_image[0]), np.max(train_image[0])

(0.0, 1.0)

In [3]:
test_image = tf.cast(test_image, tf.float32) / 255.0

np.min(test_image[0]), np.max(test_image[0])

(0.0, 1.0)

In [4]:
from tensorflow.keras.models import Sequential

input_shape = (28, 28)

model = Sequential([
  # Flatten
  layers.Flatten(input_shape=input_shape),
  # Target Layer
  layers.Dense(48, activation='relu'),
  # Output
  layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hist = model.fit(train_image, train_label, validation_data=(test_image, test_label), epochs=5, verbose=False)

In [5]:
model.evaluate(train_image, train_label, verbose=False)

[0.06548713892698288, 0.9816833138465881]

In [6]:
model.evaluate(test_image, test_label, verbose=False)

[0.09545131027698517, 0.9725000262260437]

In [7]:
loss = 'sparse_categorical_crossentropy'

def model_builder(hp):
  """
  Model Builder
  """
  model = Sequential()
  units = hp.Int('units', min_value=16, max_value=64, step=16)
  model.add(layers.Flatten(input_shape=input_shape))
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  learning_rate = hp.Choice('learning_rate', [0.001, 0.01, 0.005])
  optim = optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer=optim, loss=loss, metrics=['accuracy'])
  return model

project = 'intro_to_kt'

dirt = 'tunerfolder'; target = 'val_accuracy'

tuner = kt.Hyperband(model_builder, objective=target, max_epochs=8, directory=dirt, project_name=project)